In [2]:
import torch
import numpy as np

In [9]:
## torch.tensor 初期化
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)

np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

##_like
x_ones = torch.ones_like(x_data)

x_rand = torch.rand_like(x_data, dtype=torch.float)

print(f"Ones_like_Tensor:\n {x_ones} \n")
print(f"Random_like_Tensor:\n {x_rand} \n")

##shape指定 rand,ones,zeros
shape = (2,3,)
rand_tensor = torch.rand(shape)
print(rand_tensor)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])
Ones_like_Tensor:
 tensor([[1, 1],
        [1, 1]]) 

Random_like_Tensor:
 tensor([[0.5737, 0.8322],
        [0.0498, 0.7054]]) 

tensor([[0.4507, 0.8592, 0.0155],
        [0.9123, 0.3471, 0.2134]])


## Tensorの演算
- GPU `tensor = tensor.to('cuda')`
- numpy_like indexing,slicing,concat,mul
- inplace operation : `tensor.add_(x)`
## Torch <-> NumPy
- `n = torch.tensor.numpy()`
- `x_np = torch.from_numpy(np_array)`
- tensorとnumpyでの変更は互いに反映

## PyTorchでの自動微分


In [10]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/yumayamamoto/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100.0%


In [12]:
prediction = model(data)

In [13]:
loss = (prediction - labels).sum()
## lossもtorch型なので上の演算でmodelの計算グラフに結ばれる
loss.backward() # backward pass

In [14]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [15]:
optim.step()

## Autogradでの微分

In [21]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

In [22]:
Q = 3*a**3 - b**2


In [23]:
## 初期値 dQ/dQ　= 1
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)
print(Q)
print(a.grad)
print(b.grad)

tensor([-12.,  65.], grad_fn=<SubBackward0>)
tensor([36., 81.])
tensor([-12.,  -8.])


## freezing parameter


In [27]:
from torch import nn, optim
model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

In [28]:
#最後の層を置き換える(unfrozen)
model.fc = nn.Linear(512, 10)
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

## Neural Networks
- `torch.nn` package

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [52]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [53]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [54]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


tensor([[-0.1038, -0.0514,  0.0982, -0.0994, -0.0429, -0.0327,  0.1118, -0.0803,
          0.0123, -0.1588]], grad_fn=<AddmmBackward>)


In [55]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [56]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1917, grad_fn=<MseLossBackward>)


In [57]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0067, -0.0002,  0.0083,  0.0004, -0.0063, -0.0049])


In [58]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [59]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update